In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states

!pip install -U epiweeks
# from epiweeks import Week, Year
import epiweeks as ew

# Census API Key
from config import api_key
c = Census(api_key)

Requirement already up-to-date: epiweeks in c:\users\superpawn\anaconda3\lib\site-packages (2.1.2)


In [46]:
disease_df = pd.read_csv("diseases_us_1916_to_2011.csv")

disease_df

C:\Users\Superpawn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,epi_week,state,loc,loc_type,disease,cases,incidence_per_100000
0,196601,MN,MINNESOTA,STATE,HEPATITIS A,3,0.08
1,196601,CO,COLORADO,STATE,HEPATITIS A,1,0.05
2,196601,AZ,ARIZONA,STATE,HEPATITIS A,6,0.37
3,196601,MT,MONTANA,STATE,HEPATITIS A,2,0.28
4,196601,LA,LOUISIANA,STATE,HEPATITIS A,1,0.03
...,...,...,...,...,...,...,...
759462,194752,WI,SUPERIOR,CITY,DIPHTHERIA,0,0.00
759463,194752,FL,TAMPA,CITY,DIPHTHERIA,0,0.00
759464,194752,CO,DENVER,CITY,DIPHTHERIA,1,0.25
759465,194752,NY,BUFFALO,CITY,DIPHTHERIA,2,0.35


In [47]:
# disease_df['year'] = disease_df['epi_week'].apply(lambda x: int(x[:4]))
disease_df.dtypes

epi_week                  int64
state                    object
loc                      object
loc_type                 object
disease                  object
cases                    object
incidence_per_100000    float64
dtype: object

In [ ]:
census_data = c.acs5.get(("NAME", "B01003_001E","B19013_001E"), {'for': 'state:*'}, year=2010)

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B19013_001E": "Household Income",
                                      "NAME": "Name", "state": "State"})

census_pd = census_pd[["State", "Population", "Household Income"]]
print(len(census_pd))
census_pd.head()